In [4]:
from faceio import get_date_directories

In [5]:
video_by_day = get_date_directories('video', RATIO = 10)

In [8]:
import glob, math
import numpy as np
from tqdm import tqdm, trange
import cv2
import os
import dlib
i = 0
lastFrame = None


detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')


def getLandmarks(img):
    dets = detector(img, 1)
    for k, d in enumerate(dets):
        shape = predictor(img, d)
        currPoints = np.array([shape.part(i) for i in range(48, 68)])
        return currPoints
    dlib.hit_enter_to_continue()

def rotate(origin, point, angle):
    """
    Rotate a point counterclockwise by a given angle around a given origin.

    The angle should be given in radians.
    """
    ox, oy = origin
    px, py = point

    qx = ox + math.cos(angle) * (px - ox) - math.sin(angle) * (py - oy)
    qy = oy + math.sin(angle) * (px - ox) + math.cos(angle) * (py - oy)
    return qx, qy

def read_video(file, num):
    videos = []
    cap = cv2.VideoCapture(file)
    n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    if fps != 25:
        print('[!] WARN: Framerate inconsistent.')
    dur = n_frames/fps
    
    for i in range(num):
        _, frame = cap.read()
        if frame is None:
            frame = lastFrame
        else:
            lastFrame = frame
        videos.append(frame)
    return videos

for day in trange(len(video_by_day)):
    if day % 4 ==3:
        if not os.path.exists(video_by_day[day]['target']):
            toBeConcat = []
            # the outer loop is we enumerate each video, we need to concatenate all of them
            for index, (file, num) in enumerate(video_by_day[day]['sources']):
                # videos is images
                frames = read_video(file, num)
                # this loop is to enumerate all the pictures
                for img_num, img in enumerate(frames):
                    points = getLandmarks(img) # get landmarks return a set of points for a given image
                    # this loop is to enumerate all the pictures
                    origin_x = (points[0].x + points[6].x)/2
                    origin_y = (points[0].y + points[6].y)/2
                    origin = (origin_x, origin_y)

                    deltaY = points[6].y - points[0].y
                    deltaX = points[6].x - points[0].x
                    angle = -1 * math.atan2(deltaY, deltaX)
                    new_points = []

                    scaleFactor = 1
                    for i, _point in enumerate(points):
                        newpoint = list(rotate(origin, (_point.x, _point.y), angle))
                        if i == 0:
                            scaleFactor = -1/(newpoint[0] - origin_x)
                        newpoint[0] = (newpoint[0] - origin_x)*scaleFactor
                        newpoint[1] = (newpoint[1] - origin_y)*scaleFactor
                        new_points.append(newpoint)
                        # print(newpoint) 
                    new_points = np.concatenate(new_points).reshape(20,2)
                    toBeConcat.append(new_points)

            output = np.stack(toBeConcat)
            np.save(video_by_day[day]['target'], output)


    

100%|██████████| 120/120 [46:11<00:00, 23.10s/it]
